<a href="https://colab.research.google.com/github/AsanAlacli/JSLABS/blob/main/keyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving 3.0.0. OCR_spark_nlp_for_healthcare.txt to 3.0.0. OCR_spark_nlp_for_healthcare.txt


In [4]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

! pip install spark-nlp-display

In [5]:
import json
import os
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.0.1
Spark NLP_JSL Version : 3.0.0


In [6]:
import pandas as pd

In [9]:
pandas_df=pd.read_csv('mimic_100_pats.csv')
pandas_df

,Unnamed: 0,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
0,744,934,18589,181162.0,2191-05-10,Admission Date: [**2191-4-18**] ...
1,1315,1125,17581,144831.0,2126-05-22,Admission Date: [**2126-5-7**] Discharge ...
2,1316,1126,17581,140044.0,2127-07-29,Admission Date: [**2127-7-27**] ...
3,2117,1896,8904,147403.0,2108-03-13,Admission Date: [**2108-2-22**] ...
4,2761,2872,18054,138511.0,2147-12-11,Admission Date: [**2147-11-22**] Discharg...
...,...,...,...,...,...,...
9637,2080622,2054683,29266,102339.0,2128-07-03,Neonatology\r\r\nDoing well. remains in NCO2 f...
9638,2080623,2054684,29266,102339.0,2128-07-03,Neonatology NP Note\r\r\nPE\r\r\nswaddled in o...
9639,2080624,2054685,29266,102339.0,2128-07-03,Nursing NICU Note\r\r\n\r\r\n\r\r\n#1. Respir...
9640,2080720,2054641,29266,102339.0,2128-06-24,Neonatology Attending\r\r\nDOL 3 / PMA 40-4/7 ...


In [10]:
pandas_df.columns[0]

'Unnamed: 0'

In [11]:
pandas_df = pandas_df.rename(columns={'Unnamed: 0' : 'UKNWN_COL'})

In [12]:
pandas_df.head(20)

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
0,744,934,18589,181162.0,2191-05-10,Admission Date: [**2191-4-18**] ...
1,1315,1125,17581,144831.0,2126-05-22,Admission Date: [**2126-5-7**] Discharge ...
2,1316,1126,17581,140044.0,2127-07-29,Admission Date: [**2127-7-27**] ...
3,2117,1896,8904,147403.0,2108-03-13,Admission Date: [**2108-2-22**] ...
4,2761,2872,18054,138511.0,2147-12-11,Admission Date: [**2147-11-22**] Discharg...
5,2762,2873,18054,191467.0,2157-12-02,Admission Date: [**2157-11-25**] ...
6,3414,3704,52574,109934.0,2180-08-31,Admission Date: [**2180-8-28**] ...
7,3415,3705,52574,189995.0,2180-10-29,Admission Date: [**2180-10-19**] ...
8,3416,3706,52574,177706.0,2181-02-27,Admission Date: [**2181-2-23**] ...
9,4287,4273,11425,194366.0,2100-09-24,Admission Date: [**2100-9-5**] Discharg...


In [13]:
pandas_df.tail(20)

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
9622,2080058,2054625,29266,102339.0,2128-06-22,NPN\r\r\n\r\r\n\r\r\n#1RESP: Recv'd infant in ...
9623,2080059,2054626,29266,102339.0,2128-06-22,Respiratory Care\r\r\nInfant placed in oxyhood...
9624,2080060,2054627,29266,102339.0,2128-06-22,Neonatology Attending\r\r\nDOL 1 / PMA 40-2/7 ...
9625,2080061,2054686,29266,102339.0,2128-07-04,NICU NPN 1900-0700\r\r\n\r\r\n\r\r\nRESP O: [*...
9626,2080181,2054650,29266,102339.0,2128-06-26,npn 1900-0700\r\r\n\r\r\n\r\r\n#1 resp\r\r\nin...
9627,2080182,2054651,29266,102339.0,2128-06-26,"Neonatology Attending Note\r\r\nDay 5, PMA 40 ..."
9628,2080183,2054652,29266,102339.0,2128-06-26,NPN DAYS\r\r\n\r\r\n\r\r\nALT IN RESP:REMAINS ...
9629,2080184,2054714,29266,102339.0,2128-07-10,Attending Note\r\r\nDay of life 19 PMA 42 [**5...
9630,2080330,2054622,29266,102339.0,2128-06-21,Neonatology Attending\r\r\nFull term infant ad...
9631,2080331,2054623,29266,102339.0,2128-06-21,Neonatology Attending\r\r\n(Continued)\r\r\nym...


In [14]:
pandas_df.shape

(9642, 6)

In [15]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9642 entries, 0 to 9641
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   UKNWN_COL   9642 non-null   int64  
 1   ROW_ID      9642 non-null   int64  
 2   SUBJECT_ID  9642 non-null   int64  
 3   HADM_ID     8222 non-null   float64
 4   CHARTDATE   9642 non-null   object 
 5   TEXT        9642 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 452.1+ KB


In [16]:
pandas_df.describe()

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID
count,9.642000e+03,9.642000e+03,9642.000000,8222.000000
mean,1.098950e+06,1.099887e+06,27687.871292,151963.337509
std,5.859361e+05,5.863771e+05,24335.446171,29731.451320
min,7.440000e+02,9.340000e+02,403.000000,100481.000000
25%,6.544888e+05,6.555862e+05,11425.000000,128942.000000
50%,1.074476e+06,1.069898e+06,19823.000000,147403.000000
75%,1.622852e+06,1.625378e+06,30015.000000,177882.000000
max,2.080721e+06,2.054715e+06,99647.000000,199558.000000


In [17]:
pandas_df.sample(n=8)

,UKNWN_COL,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,TEXT
4969,1095241,1099245,75632,107794.0,2140-11-01,[**2140-11-1**] 1:09 PM\r\r\n CHEST (PORTABLE ...
2067,616119,618720,52574,177706.0,2181-02-25,[**2181-2-23**]: Operative: Cutdown of left...
6916,1558085,1534249,20745,118272.0,2157-04-07,[**2158-4-8**] 2200\r\r\n NEURO REMAINS COMB...
2002,582008,587038,56565,143505.0,2168-06-11,Coronary artery bypass graft (CABG)\r\r\n As...
5230,1150652,1159880,55921,129611.0,2179-09-14,[**2179-9-14**] 11:36 AM\r\r\n PORTABLE ABDOME...
7540,1685325,1690344,842,131102.0,2129-10-30,"Neonatology Attending\r\r\nExam AF soft, flat,..."
3516,823139,821091,13463,132415.0,2148-04-13,[**2148-4-13**] 12:10 PM\r\r\n CHEST (PORTABLE...
8291,1851432,1831514,10865,150358.0,2103-04-04,FAMILY MEETING NOTE\r\r\nFamily meeting held w...


In [18]:
pandas_df.isna().any()

UKNWN_COL     False
ROW_ID        False
SUBJECT_ID    False
HADM_ID        True
CHARTDATE     False
TEXT          False
dtype: bool

In [19]:
pandas_df.isnull().sum()

UKNWN_COL        0
ROW_ID           0
SUBJECT_ID       0
HADM_ID       1420
CHARTDATE        0
TEXT             0
dtype: int64

In [20]:
text = pandas_df.TEXT[0]

In [100]:
text = text.replace('\r', ' ')


In [101]:
text = text.replace('\n', ' ')

In [102]:
text

'Admission Date:  [**2191-4-18**]              Discharge Date:   [**2191-5-10**]      Date of Birth:  [**2112-3-14**]             Sex:   M      Service: MEDICINE      Allergies:   Ceftriaxone      Attending:[**First Name3 (LF) 2297**]   Chief Complaint:   Pneumonia      Major Surgical or Invasive Procedure:   none      History of Present Illness:   This is a 79 yo recently diagnosed with [**Doctor Last Name 6261**] syndrome   (transformation of CLL to large B cell NHL) now s/p 3 cycles of   CHOP presents with PNA diagnosed at [**Hospital1 **] ED.  The pt is   now referred from [**Hospital 478**] clinic for treatment of PNA.  The pt   presented to OSH ED on [**4-13**] and was diagnosed with LLL PNA and   was given a course of levofloxacin.  The pt did not take his abx   as prescribed.  He c/o increased SOB, DOE, and cough x 1 week.   The pt also c/o decrease appetite over past week with 4 lb   weight loss.  No increase in fatigue.  No f/c/s, n/v, d/c,   abdominal pain, chest pain, or pa

In [103]:
sparkDF= spark.createDataFrame(pandas_df)
sparkDF.show()

+---------+------+----------+--------+----------+--------------------+
|UKNWN_COL|ROW_ID|SUBJECT_ID| HADM_ID| CHARTDATE|                TEXT|
+---------+------+----------+--------+----------+--------------------+
|      744|   934|     18589|181162.0|2191-05-10|Admission Date:  ...|
|     1315|  1125|     17581|144831.0|2126-05-22|Admission Date:  ...|
|     1316|  1126|     17581|140044.0|2127-07-29|Admission Date:  ...|
|     2117|  1896|      8904|147403.0|2108-03-13|Admission Date:  ...|
|     2761|  2872|     18054|138511.0|2147-12-11|Admission Date:  ...|
|     2762|  2873|     18054|191467.0|2157-12-02|Admission Date:  ...|
|     3414|  3704|     52574|109934.0|2180-08-31|Admission Date:  ...|
|     3415|  3705|     52574|189995.0|2180-10-29|Admission Date:  ...|
|     3416|  3706|     52574|177706.0|2181-02-27|Admission Date:  ...|
|     4287|  4273|     11425|194366.0|2100-09-24|Admission Date:  ...|
|     4998|  4994|     19823|179754.0|2167-02-25|Admission Date:  ...|
|     

In [104]:
sparkDF.printSchema()

root
 |-- UKNWN_COL: long (nullable = true)
 |-- ROW_ID: long (nullable = true)
 |-- SUBJECT_ID: long (nullable = true)
 |-- HADM_ID: double (nullable = true)
 |-- CHARTDATE: string (nullable = true)
 |-- TEXT: string (nullable = true)



In [105]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = sparknlp.annotators.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

ner_tagger = MedicalNerModel()\
    .pretrained("ner_posology", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

ner_chunker = NerConverterInternal()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained("posology_re")\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]


In [106]:
model.stages

[DocumentAssembler_20c8d2fca3e4,
 SentenceDetector_ededa3cb1874,
 REGEX_TOKENIZER_7c2bd9c214ee,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 POS_6f55785005bf,
 MedicalNerModel_4a303d875127,
 NerConverterInternal_1d3509740d81,
 dependency_d5a8da6c9093,
 PosologyREModel_9839d698f661]

In [107]:
text

'Admission Date:  [**2191-4-18**]              Discharge Date:   [**2191-5-10**]      Date of Birth:  [**2112-3-14**]             Sex:   M      Service: MEDICINE      Allergies:   Ceftriaxone      Attending:[**First Name3 (LF) 2297**]   Chief Complaint:   Pneumonia      Major Surgical or Invasive Procedure:   none      History of Present Illness:   This is a 79 yo recently diagnosed with [**Doctor Last Name 6261**] syndrome   (transformation of CLL to large B cell NHL) now s/p 3 cycles of   CHOP presents with PNA diagnosed at [**Hospital1 **] ED.  The pt is   now referred from [**Hospital 478**] clinic for treatment of PNA.  The pt   presented to OSH ED on [**4-13**] and was diagnosed with LLL PNA and   was given a course of levofloxacin.  The pt did not take his abx   as prescribed.  He c/o increased SOB, DOE, and cough x 1 week.   The pt also c/o decrease appetite over past week with 4 lb   weight loss.  No increase in fatigue.  No f/c/s, n/v, d/c,   abdominal pain, chest pain, or pa

In [108]:
pd.set_option("display.max_colwidth", -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [109]:
lmodel = LightPipeline(model)

results = lmodel.fullAnnotate(text)

In [110]:
results[0].keys()

dict_keys(['sentences', 'document', 'ner_chunks', 'ner_tags', 'relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [111]:
results[0]['relations']

[Annotation(category, 1727, 1744, DURATION-DRUG, {'chunk2': 'CHOP', 'confidence': '1.0', 'entity2_end': '1744', 'chunk1': '3 cycles', 'entity2_begin': '1741', 'entity1': 'DURATION', 'entity1_begin': '1727', 'entity1_end': '1734', 'entity2': 'DRUG'}),
 Annotation(category, 5438, 5448, DRUG-ROUTE, {'chunk2': 'gtt', 'confidence': '1.0', 'entity2_end': '5448', 'chunk1': 'heparin', 'entity2_begin': '5446', 'entity1': 'DRUG', 'entity1_begin': '5438', 'entity1_end': '5444', 'entity2': 'ROUTE'}),
 Annotation(category, 5486, 5497, DOSAGE-DRUG, {'chunk2': 'pRBC', 'confidence': '1.0', 'entity2_end': '5497', 'chunk1': '2 units', 'entity2_begin': '5494', 'entity1': 'DOSAGE', 'entity1_begin': '5486', 'entity1_end': '5492', 'entity2': 'DRUG'}),
 Annotation(category, 7016, 7027, DRUG-STRENGTH, {'chunk2': '20mg', 'confidence': '1.0', 'entity2_end': '7027', 'chunk1': 'lasix', 'entity2_begin': '7024', 'entity1': 'DRUG', 'entity1_begin': '7016', 'entity1_end': '7020', 'entity2': 'STRENGTH'}),
 Annotation(

In [112]:
len(results[0]['sentences'])

90

In [113]:
results[0]['sentences'][0]

Annotation(document, 0, 550, Admission Date:  [**2191-4-18**]              Discharge Date:   [**2191-5-10**]      Date of Birth:  [**2112-3-14**]             Sex:   M      Service: MEDICINE      Allergies:   Ceftriaxone      Attending:[**First Name3 (LF) 2297**]   Chief Complaint:   Pneumonia      Major Surgical or Invasive Procedure:   none      History of Present Illness:   This is a 79 yo recently diagnosed with [**Doctor Last Name 6261**] syndrome   (transformation of CLL to large B cell NHL) now s/p 3 cycles of   CHOP presents with PNA diagnosed at [**Hospital1 **] ED., {'sentence': '0'})

In [114]:
for rel in results[0]["relations"]:
    print("{}({}={} - {}={})".format(
        rel.result, 
        rel.metadata['entity1'], 
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['chunk2']
    ))

DURATION-DRUG(DURATION=3 cycles - DRUG=CHOP)
DRUG-ROUTE(DRUG=heparin - ROUTE=gtt)
DOSAGE-DRUG(DOSAGE=2 units - DRUG=pRBC)
DRUG-STRENGTH(DRUG=lasix - STRENGTH=20mg)
DRUG-DURATION(DRUG=lasix - DURATION=x 3 over the day)
STRENGTH-DRUG(STRENGTH=40 mg - DRUG=lasix)
ROUTE-DRUG(ROUTE=IV - DRUG=lasix)
DRUG-STRENGTH(DRUG=Lipitor - STRENGTH=5 mg)
DRUG-STRENGTH(DRUG=Lipitor - STRENGTH=12.5 mg)
STRENGTH-DRUG(STRENGTH=5 mg - DRUG=atenolol)
DRUG-STRENGTH(DRUG=atenolol - STRENGTH=12.5 mg)
DRUG-ROUTE(DRUG=atenolol - ROUTE=po)
DRUG-FREQUENCY(DRUG=atenolol - FREQUENCY=qd)
DRUG-STRENGTH(DRUG=L-dopa - STRENGTH=25/100)
DRUG-FREQUENCY(DRUG=L-dopa - FREQUENCY=tid)


In [115]:
def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  return rel_df


rel_df = get_relations_df (results)

rel_df

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,DURATION-DRUG,DURATION,1727,1734,3 cycles,DRUG,1741,1744,CHOP,1.0
1,DRUG-ROUTE,DRUG,5438,5444,heparin,ROUTE,5446,5448,gtt,1.0
2,DOSAGE-DRUG,DOSAGE,5486,5492,2 units,DRUG,5494,5497,pRBC,1.0
3,DRUG-STRENGTH,DRUG,7016,7020,lasix,STRENGTH,7024,7027,20mg,1.0
4,DRUG-DURATION,DRUG,7016,7020,lasix,DURATION,7029,7044,x 3 over the day,1.0
5,STRENGTH-DRUG,STRENGTH,7289,7293,40 mg,DRUG,7298,7302,lasix,1.0
6,ROUTE-DRUG,ROUTE,7295,7296,IV,DRUG,7298,7302,lasix,1.0
7,DRUG-STRENGTH,DRUG,8490,8496,Lipitor,STRENGTH,8498,8501,5 mg,1.0
8,DRUG-STRENGTH,DRUG,8490,8496,Lipitor,STRENGTH,8513,8519,12.5 mg,1.0
9,STRENGTH-DRUG,STRENGTH,8498,8501,5 mg,DRUG,8504,8511,atenolol,1.0


In [116]:
len(rel_df)

15

In [117]:
from sparknlp_display import RelationExtractionVisualizer


vis = RelationExtractionVisualizer()
vis.display(results[0], 'relations', show_relations=True) # default show_relations: True

/usr/local/lib/python3.7/dist-packages/sparknlp_display/relation_extraction.py:354: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  relation_coordinates = np.array(relation_coordinates)


In [118]:
results[0]['ner_tags']

[Annotation(named_entity, 0, 8, O, {'word': 'Admission', 'confidence': '0.9992'}),
 Annotation(named_entity, 10, 13, O, {'word': 'Date', 'confidence': '0.8498'}),
 Annotation(named_entity, 14, 14, O, {'word': ':', 'confidence': '0.9955'}),
 Annotation(named_entity, 17, 31, O, {'word': '[**2191-4-18**]', 'confidence': '0.8852'}),
 Annotation(named_entity, 46, 54, O, {'word': 'Discharge', 'confidence': '0.9319'}),
 Annotation(named_entity, 56, 59, O, {'word': 'Date', 'confidence': '0.6157'}),
 Annotation(named_entity, 60, 60, O, {'word': ':', 'confidence': '0.9878'}),
 Annotation(named_entity, 64, 78, O, {'word': '[**2191-5-10**]', 'confidence': '0.7824'}),
 Annotation(named_entity, 85, 88, O, {'word': 'Date', 'confidence': '0.633'}),
 Annotation(named_entity, 90, 91, O, {'word': 'of', 'confidence': '0.6655'}),
 Annotation(named_entity, 93, 97, O, {'word': 'Birth', 'confidence': '0.7505'}),
 Annotation(named_entity, 98, 98, O, {'word': ':', 'confidence': '0.974'}),
 Annotation(named_enti

# Clinical RE

In [119]:
clinical_ner_tagger = MedicalNerModel()\
    .pretrained("ner_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setRelationPairs(["problem-test", "problem-treatment"]) # we can set the possible relation pairs (if not set, all the relations will be calculated)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    clinical_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
re_clinical download started this may take some time.
Approximate size to download 6 MB
[OK!]


In [120]:
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df1 = get_relations_df (annotations)

rel_df1[rel_df1.relation!="O"]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
8,TrAP,PROBLEM,1163,1165,CVA,TREATMENT,1176,1185,ASA/plavix,0.86630833
11,TeRP,TEST,1272,1282,bronchscopy,PROBLEM,1323,1337,malignant cells,0.996745
12,TeRP,TEST,1549,1559,Lung biopsy,PROBLEM,1580,1596,a large Bcell NHL,0.99402547
13,TeRP,PROBLEM,2165,2185,decreased attenuation,TEST,2206,2219,the prior exam,0.99115014
15,TeRP,PROBLEM,2567,2595,stomach and intestinal cancer,TEST,2669,2670,VS,0.9999999
22,TeRP,PROBLEM,2754,2787,conjunctivae anicteric/noninjected,TEST,2814,2816,JVP,0.999997
28,TeRP,PROBLEM,5319,5325,hypoxic,TEST,5338,5340,CXR,0.99431425
35,TeRP,PROBLEM,5544,5559,flash pulm edema,TREATMENT,5570,5578,intubated,0.99085766
38,TrAP,PROBLEM,5828,5835,diarrhea,TEST,5858,5866,a lactate,0.58548725
40,TeRP,PROBLEM,5911,5939,hypoxic respiratory failure,TREATMENT,5945,5953,intubated,0.9862671


In [121]:
events_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

clinical_re_Model = RelationExtractionModel()\
    .pretrained("re_temporal_events_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(4)\
    .setPredictionThreshold(0.9)

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
re_temporal_events_clinical download started this may take some time.
Approximate size to download 5.9 MB
[OK!]


In [122]:
events_ner_tagger.getClasses()

['O',
 'B-OCCURRENCE',
 'B-DATE',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-DURATION',
 'I-DURATION',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'B-TREATMENT',
 'I-TREATMENT',
 'I-DATE',
 'B-TEST',
 'I-TEST',
 'B-CLINICAL_DEPT',
 'I-CLINICAL_DEPT',
 'B-FREQUENCY',
 'I-FREQUENCY',
 'I-EVIDENTIAL',
 'B-TIME',
 'I-TIME']

In [123]:
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df2 = get_relations_df (annotations)

rel_df2 = rel_df2[(rel_df2.relation!="O")]

rel_df2

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,OVERLAP,OCCURRENCE,0,8,Admission,DATE,17,31,[**2191-4-18**],1.0
1,OVERLAP,OCCURRENCE,0,8,Admission,OCCURRENCE,46,54,Discharge,1.0
2,OVERLAP,DATE,17,31,[**2191-4-18**],OCCURRENCE,46,54,Discharge,0.9914493
3,OVERLAP,TREATMENT,179,189,Ceftriaxone,PROBLEM,255,263,Pneumonia,0.9547989
4,OVERLAP,OCCURRENCE,222,223,LF,PROBLEM,255,263,Pneumonia,0.99990416
...,...,...,...,...,...,...,...,...,...,...
262,OVERLAP,OCCURRENCE,8477,8485,Admission,TREATMENT,8504,8511,atenolol,0.9998721
263,OVERLAP,TREATMENT,8490,8496,Lipitor,TREATMENT,8504,8511,atenolol,1.0
264,OVERLAP,TREATMENT,8504,8511,atenolol,FREQUENCY,8524,8525,qd,1.0
265,OVERLAP,TREATMENT,8528,8533,L-dopa,FREQUENCY,8542,8544,tid,1.0


In [124]:
annotations = lmodel.fullAnnotate(text)

rel_df3 = get_relations_df (annotations)

rel_df3.confidence = rel_df3.confidence.astype(float)

rel_df3 = rel_df3[(rel_df3.relation!="O")]

rel_df3[(rel_df3.relation!="O")&(rel_df3.entity1!=rel_df3.entity2)]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,OVERLAP,OCCURRENCE,0,8,Admission,DATE,17,31,[**2191-4-18**],1.000000
2,OVERLAP,DATE,17,31,[**2191-4-18**],OCCURRENCE,46,54,Discharge,0.991449
3,OVERLAP,TREATMENT,179,189,Ceftriaxone,PROBLEM,255,263,Pneumonia,0.954799
4,OVERLAP,OCCURRENCE,222,223,LF,PROBLEM,255,263,Pneumonia,0.999904
5,OVERLAP,OCCURRENCE,375,383,diagnosed,PROBLEM,390,398,[**Doctor,1.000000
...,...,...,...,...,...,...,...,...,...,...
261,OVERLAP,OCCURRENCE,8477,8485,Admission,TREATMENT,8490,8496,Lipitor,0.999998
262,OVERLAP,OCCURRENCE,8477,8485,Admission,TREATMENT,8504,8511,atenolol,0.999872
264,OVERLAP,TREATMENT,8504,8511,atenolol,FREQUENCY,8524,8525,qd,1.000000
265,OVERLAP,TREATMENT,8528,8533,L-dopa,FREQUENCY,8542,8544,tid,1.000000


# Admission-Discharge Events Exraction

In [125]:
events_admission_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_admission_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")    

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_admission_ner_tagger,
    ner_chunker,
    dependency_parser,
    clinical_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_admission_clinical download started this may take some time.
Approximate size to download 14.5 MB
[OK!]


In [126]:
events_admission_ner_tagger.getClasses()

['O',
 'B-EVIDENTIAL',
 'I-OCCURRENCE',
 'I-TREATMENT',
 'I-DATE',
 'B-ADMISSION',
 'B-TREATMENT',
 'I-TIME',
 'I-FREQUENCY',
 'B-FREQUENCY',
 'B-OCCURRENCE',
 'I-CLINICAL_DEPT',
 'B-DURATION',
 'I-PROBLEM',
 'I-TEST',
 'B-DATE',
 'B-TIME',
 'I-EVIDENTIAL',
 'I-DURATION',
 'B-DISCHARGE',
 'B-TEST',
 'B-PROBLEM',
 'B-CLINICAL_DEPT']

In [127]:
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df4 = get_relations_df (annotations)

rel_df4.confidence = rel_df4.confidence.astype(float)

rel_df4[(rel_df4.relation!="O")]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,OVERLAP,ADMISSION,0,8,Admission,DISCHARGE,46,54,Discharge,1.000000
1,BEFORE,OCCURRENCE,375,383,diagnosed,PROBLEM,418,425,syndrome,0.999966
2,OVERLAP,PROBLEM,418,425,syndrome,PROBLEM,448,470,CLL to large B cell NHL,0.987992
3,OVERLAP,FREQUENCY,481,488,3 cycles,TREATMENT,495,498,CHOP,1.000000
4,BEFORE,FREQUENCY,481,488,3 cycles,EVIDENTIAL,500,507,presents,0.911655
...,...,...,...,...,...,...,...,...,...,...
267,OVERLAP,DISCHARGE,8588,8596,Discharge,DISCHARGE,8626,8634,Discharge,1.000000
268,OVERLAP,DISCHARGE,8588,8596,Discharge,DISCHARGE,8662,8670,Discharge,1.000000
269,OVERLAP,DISCHARGE,8626,8634,Discharge,DISCHARGE,8662,8670,Discharge,1.000000
270,OVERLAP,DISCHARGE,8626,8634,Discharge,DISCHARGE,8698,8706,Discharge,1.000000


# Test-Result-Date Extraction

In [128]:
trd_ner_tagger = MedicalNerModel()\
    .pretrained("jsl_ner_wip_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")       

trd_re_Model = RelationExtractionModel()\
    .pretrained("re_test_result_date", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    trd_ner_tagger,
    ner_chunker,
    dependency_parser,
    trd_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

jsl_ner_wip_clinical download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
re_test_result_date download started this may take some time.
Approximate size to download 9.3 MB
[OK!]


In [129]:
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df5 = get_relations_df (annotations)

rel_df5.confidence = rel_df5.confidence.astype(float)

rel_df5[(rel_df5.relation!="O")]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,is_result_of,Admission_Discharge,0,8,Admission,Admission_Discharge,46,54,Discharge,0.989884
2,is_finding_of,Admission_Discharge,0,8,Admission,Disease_Syndrome_Disorder,255,263,Pneumonia,0.986574
3,is_finding_of,Admission_Discharge,0,8,Admission,Age,360,364,79 yo,0.587797
4,is_finding_of,Admission_Discharge,0,8,Admission,Oncological,430,450,transformation of CLL,0.986750
5,is_finding_of,Admission_Discharge,0,8,Admission,Oncological,455,470,large B cell NHL,0.958758
...,...,...,...,...,...,...,...,...,...,...
1238,is_result_of,Frequency,8542,8544,tid,Admission_Discharge,8626,8634,Discharge,0.999968
1239,is_result_of,Frequency,8542,8544,tid,Admission_Discharge,8662,8670,Discharge,0.979969
1240,is_result_of,Frequency,8542,8544,tid,Admission_Discharge,8698,8706,Discharge,0.720760
1241,is_finding_of,Admission_Discharge,8588,8596,Discharge,Admission_Discharge,8626,8634,Discharge,0.997754


In [130]:
events_ner_tagger = MedicalNerModel()\
    .pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")     

events_re_ner_chunk_filter = RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")
    
events_re_Model = RelationExtractionDLModel() \
    .pretrained('redl_temporal_events_biobert', "en", "clinical/models")\
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setOutputCol("relations")

pipeline = Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    events_ner_tagger,
    ner_chunker,
    dependency_parser,
    events_re_ner_chunk_filter,
    events_re_Model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
redl_temporal_events_biobert download started this may take some time.
Approximate size to download 383.3 MB
[OK!]


In [131]:
lmodel = LightPipeline(model)
annotations = lmodel.fullAnnotate(text)

rel_df6 = get_relations_df (annotations)

rel_df6.confidence = rel_df6.confidence.astype(float)

rel_df6 = rel_df6[(rel_df6.relation!="O")]

rel_df6[(rel_df6.relation!="O")&(rel_df6.entity1!=rel_df6.entity2)]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,OVERLAP,OCCURRENCE,0,8,Admission,DATE,17,31,[**2191-4-18**],0.626051
2,OVERLAP,OCCURRENCE,0,8,Admission,TREATMENT,179,189,Ceftriaxone,0.681332
4,OVERLAP,OCCURRENCE,0,8,Admission,PROBLEM,255,263,Pneumonia,0.716139
6,OVERLAP,OCCURRENCE,0,8,Admission,PROBLEM,418,425,syndrome,0.503962
7,OVERLAP,OCCURRENCE,0,8,Admission,FREQUENCY,481,488,3 cycles,0.532811
...,...,...,...,...,...,...,...,...,...,...
148,OVERLAP,TREATMENT,8462,8472,Medications,OCCURRENCE,8477,8485,Admission,0.675706
151,OVERLAP,TREATMENT,8462,8472,Medications,FREQUENCY,8524,8525,qd,0.802994
153,OVERLAP,TREATMENT,8462,8472,Medications,FREQUENCY,8542,8544,tid,0.723612
154,OVERLAP,TREATMENT,8462,8472,Medications,OCCURRENCE,8762,8768,Expired,0.619881


In [132]:
annotations = lmodel.fullAnnotate(text)

rel_df7 = get_relations_df (annotations)

rel_df7.confidence = rel_df7.confidence.astype(float)

rel_df7[(rel_df7.relation!="O")]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,OVERLAP,OCCURRENCE,0,8,Admission,DATE,17,31,[**2191-4-18**],0.645524
1,OVERLAP,OCCURRENCE,0,8,Admission,OCCURRENCE,46,54,Discharge,0.534560
2,OVERLAP,OCCURRENCE,0,8,Admission,TREATMENT,179,189,Ceftriaxone,0.534865
3,OVERLAP,OCCURRENCE,0,8,Admission,OCCURRENCE,222,223,LF,0.749727
4,OVERLAP,OCCURRENCE,0,8,Admission,PROBLEM,255,263,Pneumonia,0.785085
...,...,...,...,...,...,...,...,...,...,...
155,OVERLAP,TREATMENT,8462,8472,Medications,FREQUENCY,8524,8525,qd,0.686098
156,OVERLAP,TREATMENT,8462,8472,Medications,TREATMENT,8528,8533,L-dopa,0.803257
157,OVERLAP,TREATMENT,8462,8472,Medications,FREQUENCY,8542,8544,tid,0.579104
158,OVERLAP,TREATMENT,8462,8472,Medications,OCCURRENCE,8762,8768,Expired,0.622821


In [133]:
from sparknlp.pretrained import PretrainedPipeline

In [134]:
era_pipeline = PretrainedPipeline('explain_clinical_doc_era', 'en', 'clinical/models')

explain_clinical_doc_era download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [135]:
era_pipeline.model.stages

[DocumentAssembler_81ef1f17c7c1,
 SentenceDetector_0b67d45c215f,
 REGEX_TOKENIZER_4d38514cc549,
 POS_6f55785005bf,
 dependency_d5a8da6c9093,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_7cb29c8c904c,
 NerConverter_dc8e863a00ea,
 RelationExtractionModel_14b00157fc1a,
 ASSERTION_DL_25881ab6309e]

In [136]:
annotations = era_pipeline.annotate(text)

annotations.keys()

dict_keys(['sentences', 'clinical_ner_tags', 'document', 'clinical_ner_chunks', 'assertion', 'clinical_relations', 'tokens', 'embeddings', 'pos_tags', 'dependencies'])

In [137]:
result = era_pipeline.fullAnnotate(text)[0]

In [138]:
import pandas as pd

chunks=[]
entities=[]
begins=[]
ends=[]

for n in result['clinical_ner_chunks']:
    
    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity']) 
        
df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities})

df

,chunks,begin,end,entities
0,Admission,0,8,OCCURRENCE
1,[**2191-4-18**],17,31,DATE
2,Discharge,46,54,OCCURRENCE
3,Ceftriaxone,179,189,TREATMENT
4,LF,222,223,OCCURRENCE
...,...,...,...,...
238,qd,8524,8525,FREQUENCY
239,L-dopa,8528,8533,TREATMENT
240,tid,8542,8544,FREQUENCY
241,Expired,8762,8768,OCCURRENCE


In [139]:
df.head(40)

,chunks,begin,end,entities
0,Admission,0,8,OCCURRENCE
1,[**2191-4-18**],17,31,DATE
2,Discharge,46,54,OCCURRENCE
3,Ceftriaxone,179,189,TREATMENT
4,LF,222,223,OCCURRENCE
5,Pneumonia,255,263,PROBLEM
6,diagnosed,375,383,OCCURRENCE
7,[**Doctor,390,398,PROBLEM
8,syndrome,418,425,PROBLEM
9,CLL,448,450,PROBLEM


In [140]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  rel_df.confidence = rel_df.confidence.astype(float)
  
  return rel_df

In [141]:
annotations = era_pipeline.fullAnnotate(text)

rel_df8 = get_relations_df (annotations, 'clinical_relations')

rel_df8

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,OVERLAP,OCCURRENCE,0,8,Admission,DATE,17,31,[**2191-4-18**],1.000000
1,OVERLAP,OCCURRENCE,0,8,Admission,OCCURRENCE,46,54,Discharge,1.000000
2,AFTER,OCCURRENCE,0,8,Admission,TREATMENT,179,189,Ceftriaxone,0.772236
3,OVERLAP,OCCURRENCE,0,8,Admission,OCCURRENCE,222,223,LF,0.837940
4,OVERLAP,OCCURRENCE,0,8,Admission,PROBLEM,255,263,Pneumonia,0.551889
...,...,...,...,...,...,...,...,...,...,...
809,OVERLAP,TREATMENT,8528,8533,L-dopa,OCCURRENCE,8762,8768,Expired,0.969981
810,BEFORE,TREATMENT,8528,8533,L-dopa,DATE,8788,8805,by:[**2191-5-19**],0.984598
811,OVERLAP,FREQUENCY,8542,8544,tid,OCCURRENCE,8762,8768,Expired,1.000000
812,BEFORE,FREQUENCY,8542,8544,tid,DATE,8788,8805,by:[**2191-5-19**],0.974254


In [146]:
text

'Admission Date:  [**2191-4-18**]              Discharge Date:   [**2191-5-10**]      Date of Birth:  [**2112-3-14**]             Sex:   M      Service: MEDICINE      Allergies:   Ceftriaxone      Attending:[**First Name3 (LF) 2297**]   Chief Complaint:   Pneumonia      Major Surgical or Invasive Procedure:   none      History of Present Illness:   This is a 79 yo recently diagnosed with [**Doctor Last Name 6261**] syndrome   (transformation of CLL to large B cell NHL) now s/p 3 cycles of   CHOP presents with PNA diagnosed at [**Hospital1 **] ED.  The pt is   now referred from [**Hospital 478**] clinic for treatment of PNA.  The pt   presented to OSH ED on [**4-13**] and was diagnosed with LLL PNA and   was given a course of levofloxacin.  The pt did not take his abx   as prescribed.  He c/o increased SOB, DOE, and cough x 1 week.   The pt also c/o decrease appetite over past week with 4 lb   weight loss.  No increase in fatigue.  No f/c/s, n/v, d/c,   abdominal pain, chest pain, or pa